# Normal vs Leveraged Gains

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display, Markdown, Latex

## Gain Generator

This is a function that produces a series of random daily gains. The default parameters produces a cummulative gain of 10% (`target = 1.1`) over 200 days (`count = 200`), with a daily standard deviation 2% (`m = 0.02`). This closely resembles the SP-500. Users may also choose a uniform distribution by setting the `normal` keyword to be `False`.

In [ ]:
def gain_generator(target=1.1, count=200, m=0.02, verbose=0, normal=True):
    if normal:
        # Normally distributed numbers
        d = m * np.random.normal(size=count)
    else:
        # Uniform random numbers in between [-m, +m]
        d = 2.0 * m * np.random.random(count) - m
    # Force day 0 to be 0
    d[0]= 0.0
    # Find a gain that produces desired cumulative gain at the end of the period
    k = 0
    offset = 0;
    x = np.ones((count))
    while k < 20:
        x[1:] = 1.0 + d[1:] + offset
        ret = np.prod(x)
        delta = np.prod(x) - target
        if verbose:
            print('{:3d}  o = {:.6f} -> {:.6f}  {:+9.6f}'.format(k, offset, ret, delta))
        if abs(delta) < 1.0e-5:
            break
        offset -= 1.0 / count * delta
        k += 1
    return x - 1.0

In [ ]:
g = gain_generator()
np.std(g)

In [ ]:
matplotlib.rcParams['font.sans-serif'] = "Arial"
matplotlib.rcParams['font.family'] = "sans-serif"

## Simulation

This simulation emulates a random daily return with the targeted compound gain and a 3x leveraged return

In [ ]:
# For a typical growth, low volatility, e.g., SPY, daily fluction can be higher
# m = 0.01; gain = 1.1; count = 200

# For high growth, high volatility, e.g., TQQQ, daily fluction can be higher
# m = 0.018; gain = 1.08; count = 100
m = 0.018; gain = 1.05; count = 50
# m = 0.018; gain = 1.08; count = 25

# Simulation
g = gain_generator(gain, count=count, m=m)
t = np.arange(count)
n = np.prod(1.0 + g)
l = np.prod(1.0 + 3.0 * g)
cn = 100 * (np.cumprod(1 + g) - 1)
cl = 100 * (np.cumprod(1 + 3.0 * g) - 1)
clr1 = 'tab:blue'
clr2 = 'tab:green'
delta = 0.5

# Distribution of the Gains
plt.figure(dpi=120, figsize=(8, 3))
b = np.arange(-12 - 0.5 * delta, 12 + delta, delta)
n, _, patches = plt.hist(300 * g, bins=b, facecolor='green', edgecolor='k', alpha=0.5)
for b, p in zip(b, patches):
    if b < -0.5 * delta:
        p.set_facecolor('red')
    elif b < 0.5 * delta:
        p.set_facecolor('grey')
plt.text(-16, 0.96 * max(n), 'Up Days = {}\nDown Days = {}\nTotal Days = {}'.format(
    np.sum(g >= 0), np.sum(g < 0), count), linespacing=1.8, va='top')
plt.grid(color='black', alpha=0.15)
plt.xlim((-17, 17))
plt.xlabel('Gain (%)')
plt.ylabel('Count')
plt.title('Distribution of Daily Leveraged Gains ({:.1f}% bins)'.format(delta), fontweight=800)

# Raw Gain Values
plt.figure(dpi=120, figsize=(8, 3))
if len(t) <= 50:
    marker = '.'
    markersize = 8
else:
    marker = ''
plt.plot(t, 100.0 * g, marker=marker, markersize=markersize, color=clr1, label='Normal')
plt.plot(t, 300.0 * g, marker=marker, markersize=markersize, color=clr2, label='Leveraged')
plt.grid(color='black', alpha=0.15)
plt.xlim((-0.04 * count, 1.14 * count))
plt.ylim((-16, 16))
plt.xlabel('Days')
plt.ylabel('Gain (%)')
plt.legend()
plt.title('Daily Gains', fontweight=800)

# Cumulative Gain Over the Period
plt.figure(dpi=120, figsize=(8, 3))
plt.plot(t, cn, label='Normal', color=clr1)
plt.plot(t, cl, label='Leveraged', color=clr2)
plt.plot(t[-1], cn[-1], marker='o', color=clr1)
plt.text(t[-1] * 1.03, cn[-1], '{:.2f}%'.format(cn[-1]), color=clr1, va='center')
plt.plot(t[-1], cl[-1], marker='o', color=clr2)
plt.text(t[-1] * 1.03, cl[-1], '{:.2f}%'.format(cl[-1]), color=clr2, va='center')
plt.grid(color='black', alpha=0.15)
plt.xlim((-0.04 * count, 1.14 * count))
plt.xlabel('Days')
plt.ylabel('Gain (%)')
plt.legend(loc='upper left')
plt.title('Cummulative Gain After {} Days'.format(count), fontweight=800)

print('')

In [ ]:
plt.close()

## Multiple Realizations

In [ ]:
# Number of realizations
r = 20

# Table header
output  = '| \# | Up Days | Normal | Leveraged | Extremes | Leveraged > Normal |\n'
output += '| --- | --: | --: | --: | --: | :-: |\n'

# Now we emulate a bunch of runs
uu = 0
nn = 0
ll = 0
ee = (0, 0)
lab = {True: '+', False: ' '}
for k in range(1, r + 1):
    g = gain_generator(gain, count, m=m)
    u = np.sum(g >= 0)
    n = np.prod(1.0 + g)
    l = np.prod(1.0 + 3.0 * g)
    h = np.cumprod(1.0 + 3.0 * g)
    e = [100 * (x - 1) for x in (np.min(h), np.max(h))]
    ee = (min(ee[0], e[0]), max(ee[1], e[1]))
    uu += u
    nn += n
    ll += l
    output += '| {:2d} | {:3d} ({:.1f}%) | {:+.2f}% | {:+.2f}% | ({:+.1f}%, {:+.1f}%) | {} |\n'.format(
        k, u, 100 * u / count, 100 * (n - 1.0), 100 * (l - 1.0), e[0], e[1], lab[l > n])
nn /= r
ll /= r
# output += '\n'

# output += '| --- | --: | --: | --: | --: | :-: |\n'
output += '| **Avg** | **{:.1f} ({:.1f}%)** | **{:+.2f}%** | **{:+.2f}%** | **({:+5.1f}%, {:+5.1f}%)** | **{}** |\n'.format(
    uu / r, 100 * uu / count / r, 100 * (nn - 1), 100 * (ll - 1), ee[0], ee[1], lab[ll > nn])

display(Markdown(output))

display(Markdown('## #Average from {} realizations:'.format(r)))
display(Markdown('Up days = {:.1f} ({:.1f}%)   normal = {:+.2f}%   leveraged = {:+.2f}%   extremes = ({:+5.1f}%, {:+5.1f}%)'.format(
    uu / r, 100 * uu / count / r, 100 * (nn - 1), 100 * (ll - 1), ee[0], ee[1])))